# Rocq-ml-toolbox

The goal of this notebook is to present the very basic steps to create and interact with a custom build docker image.

## Installation

First, let's start by installing all the requirements.
In this notebook, we will **USE** docker.

First build the base docker images:

In `docker/`, do
`docker compose build rocq-server`.

In your python env

```
pip install git+https://github.com/llm4rocq/pytanque.git
pip install "rocq-ml-toolbox[all] @ git+https://github.com/llm4rocq/rocq-ml-toolbox.git"
```


## Docker image

Let's load an Opam config.

In [ ]:
from rocq_ml_toolbox.docker.opam_docker import OpamConfig

config = OpamConfig.from_yaml('coq-corn.yaml')

Now, we can construct the associated docker image.

In [ ]:

import docker
from rocq_ml_toolbox.docker.opam_docker import OpamDocker

client = docker.from_env()
opam_docker = OpamDocker(config)

opam_docker.install_project(" ".join(config.packages))

Now, if you want to save it:

In [ ]:
opam_docker.container.commit(config.name, config.tag)

Let's try to close it, and reload it again:

In [ ]:
opam_docker.close()
opam_docker = OpamDocker(config)

# Do we need to install packages a second time?
opam_docker.install_project(" ".join(config.packages))

Now let's do our usual tasks:

In [ ]:
from typing import Dict, List
from collections import defaultdict

from tqdm import tqdm
from rocq_ml_toolbox.inference import PetClient
from rocq_ml_toolbox.parser.rocq_parser import RocqParser, Source

opam_docker.start_inference_server()
client = PetClient('127.0.0.1', 5000)
client.connect()
parser = RocqParser(client)

# we can extract all source files inside the opam env:
output_sources: Dict[str, List[Source]] = defaultdict(list)
for lib_name in tqdm(opam_docker.list_opam_folder(), desc='Extract sources'):
    output_sources[lib_name] = opam_docker.extract_source_files_from_folder(lib_name)


for lib_name, sources in tqdm(output_sources.items(), desc='Extract TOC'):
    for source in sources:
        toc = parser.extract_toc(source)
        print(toc)
        input('continue?')